In [2]:
#Maksimov Andrey MAEE'19

In [3]:
#Home Work 4

In [4]:
from collections import Counter
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
import numpy as np
import matplotlib
import zipfile
import math
import xml
import sys
import os

In [5]:
import csv
import requests #Library for HTTP requests

In [6]:
#Task1
#Обратимся к архиву с данными, затем с помощью функции open() прочитаем необходимые нам файлы(есть согласно условию)
r = requests.get('https://drive.google.com/uc?export=download&id=0B6ZlG_Eygdj-c1kzcmUxN05VUXM')
with open('Directory.zip', 'wb') as code:
    code.write(r.content)
path = os.getcwd() + '/Directory.zip'
zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall(os.getcwd())
zip_ref.close()

In [7]:
data = pd.DataFrame.from_csv(path='survey_results_public.csv', sep = ',', header=0, encoding='utf-8')
data.iloc[:,:1].head()

,Professional
Respondent,
1,Student
2,Student
3,Professional developer
4,Professional non-developer who sometimes write...
5,Professional developer


In [8]:
schema = pd.DataFrame.from_csv(path='survey_results_schema.csv', sep = ',', header=0, encoding='utf-8')
schema.head()

,Question
Column,
Respondent,Respondent ID number
Professional,Which of the following best describes you?
ProgramHobby,Do you program as a hobby or contribute to ope...
Country,In which country do you currently live?
University,"Are you currently enrolled in a formal, degree..."


In [9]:
#Task2
data_country = data['Country'].value_counts()
data_country.head(10)

United States         11455
India                  5197
United Kingdom         4395
Germany                4143
Canada                 2233
France                 1740
Poland                 1290
Australia               913
Russian Federation      873
Spain                   864
Name: Country, dtype: int64

In [10]:
data_result = data['Country'].value_counts() / len(data['Country'])
data_result.head(10)

United States         0.222895
India                 0.101125
United Kingdom        0.085519
Germany               0.080616
Canada                0.043450
France                0.033857
Poland                0.025101
Australia             0.017765
Russian Federation    0.016987
Spain                 0.016812
Name: Country, dtype: float64

In [11]:
#Task3
from xml.etree import ElementTree
from bs4 import BeautifulSoup
import re

WIKI_URL = "https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population"

r = requests.get(WIKI_URL)
soup = BeautifulSoup(r.content, 'lxml') 
table_classes = {"class": ["sortable", "plainrowheaders"]}
wikitables = soup.findAll("table", table_classes)

tmp_find = soup.findAll("td")

#Данная часть кода показывает нам, как выглядят необходимые блоки с городами, что позволяет нам правильно
#распарсить страницу

#for string in tmp_find:
#    print(string.prettify())

number_of_string = []
for string in tmp_find:
    l = str(string)
    number_of_string.append(l)
    
#Теперь мы будем идти по тегам - по их началу и выбирать те, после которых содержатся страны
i = 0
result_list = []
result_population = []
while i != len(number_of_string) - 1:
    if(number_of_string[i].count('a href="/wiki/') != 0):
        if(number_of_string[i].find('(') != -1):
            number_of_string[i] = number_of_string[i][0:number_of_string[i].find('(')]
        result_list.append(number_of_string[i])
        result_list.append(number_of_string[i + 1])
        result_population.append(number_of_string[i + 1])
        i = i + 1
    else:
        i = i + 1
        
result_list.pop(0) 
result_population.pop(0) #Строки с количеством населения
result_word = [] #Строки со странами

for i in range(0, len(result_list)):
    for word in (result_list[i].split()):
        if word.count('href="/wiki/') != 0:
            while word.find('_') != -1:
                index = word.find('_')
                word = word[0:index] + ' ' + word[index + 1:]
            result_word.append(word)

#P.S Если данная задача будет разобрана в классе и будет рассказано, как парсить деревья с очень высокой вложенностью
#тегов, при условии их совпадения, будет очень здорово!
#P.P.S (( Разбор был, но переписывать жалко что-то

words = [] #Страны
for word in result_word:
    if(word.count('List') != 0):
        break
    if(word.count('File') == 0):
        word = word[12:len(word) - 1]
        if(word[0] == 'R'):
            if (word.count('Republic') != 0):
                i = word.rindex(' ')
                word = word[i + 1: ]
        words.append(word)
        
#words.pop(len(words) - 1)
    
population = [] #Количество населения
for count in result_population:
    if(count.count('class') != 0):
        break
    count = count.replace(',','')
    population.append(float(count[4:len(count) - 5]))
    
add_frame = pd.DataFrame({'Population': population, 'Country_' : words}, index = None)
add_frame

,Country_,Population
0,China,1.389150e+09
1,India,1.327720e+09
2,United States,3.266040e+08
3,Indonesia,2.618909e+08
4,Pakistan,2.105420e+08
5,Brazil,2.086420e+08
6,Nigeria,1.933925e+08
7,Bangladesh,1.639720e+08
8,Russia,1.468771e+08
9,Japan,1.265900e+08


In [12]:
respondents = data['Country'].value_counts()
respondents_frame = pd.DataFrame({'Country_': respondents.index, 'Country': pd.to_numeric(respondents.values)}, index = None)

tmp_frame = pd.merge(add_frame, respondents_frame, on='Country_', how='inner')
tmp_frame = tmp_frame[tmp_frame['Country'] >= 100]

tmp_frame['Ratio'] = tmp_frame['Country'] / tmp_frame['Population']

tmp_frame = tmp_frame.sort_values(by = ['Ratio'], ascending = False)
tmp_frame.head(10)

,Country_,Population,Country,Ratio
130,Slovenia,2065895.0,303,0.000147
112,Ireland,4792500.0,345,0.000072
92,Switzerland,8465234.0,595,0.000070
19,United Kingdom,65648000.0,4395,0.000067
91,Israel,8815060.0,575,0.000065
123,Lithuania,2810118.0,176,0.000063
84,Sweden,10112669.0,611,0.000060
35,Canada,37013700.0,2233,0.000060
90,Austria,8823054.0,477,0.000054
107,Finland,5509984.0,287,0.000052


In [13]:
#Task4

#Выведем необходимую колонку, предварительно ее отсортировав
data['VersionControl'].value_counts()

Git                                            21266
Subversion                                      2790
Team Foundation Server                          2255
I don't use version control                     1468
I use some other system                          924
Zip file back-ups                                609
Mercurial                                        591
Copying and pasting files to network shares      510
Visual Source Safe                               196
Rational ClearCase                               121
Name: VersionControl, dtype: int64

In [14]:
#Task5

languages = [] 

#Посмотрим на состав наших данных
respond_language = data['HaveWorkedLanguage'].dropna()

#Будем проходиться, по данным, которые предоставил каждый респондент и доставать все указанные языки программирования
for new_resp_language in respond_language:
    set_of_resp_language = new_resp_language.split(';')
    for set_lang in set_of_resp_language:
        set_lang = set_lang.replace(' ','')
        if languages.count(set_lang) == 0:
            languages.append(set_lang)
            
print(languages)

['Swift', 'JavaScript', 'Python', 'Ruby', 'SQL', 'Java', 'PHP', 'Matlab', 'R', 'Rust', 'CoffeeScript', 'Clojure', 'Elixir', 'Erlang', 'Haskell', 'C#', 'Objective-C', 'C', 'C++', 'Assembly', 'VB.NET', 'Perl', 'Scala', 'F#', 'TypeScript', 'Lua', 'VBA', 'Groovy', 'Go', 'Smalltalk', 'VisualBasic6', 'CommonLisp', 'Dart', 'Julia', 'Hack']


In [15]:
#Task6

language_series = pd.Series(0, index = languages)
#Создадим Series, в которой будем для каждого языка суммировать количество относящихся респондентов

for new_resp_language in respond_language:
    set_of_resp_language = new_resp_language.split(';')
    for set_lang in set_of_resp_language:
        set_lang = set_lang.replace(' ','')
        language_series[set_lang] = language_series[set_lang] + 1
        
language_series = language_series.sort_values(ascending = False)
language_series.head(10)

JavaScript    22875
SQL           18754
Java          14524
C#            12476
Python        11704
PHP           10290
C++            8155
C              6974
TypeScript     3488
Ruby           3324
dtype: int64

In [16]:
#Task7
import time

def isnan(value): 
    try: 
        return math.isnan(float(value)) 
    except: 
        return False

#Получим указанные языки и страны, которые к ним относятся
respond_language = data['HaveWorkedLanguage']
respond_language = respond_language.get_values()

data_result = data['Country']
data_result = data_result.get_values()

tmp_dict = dict.fromkeys(set(data_result))
for country in tmp_dict:
    tmp_dict[country] = list()
        
#Теперь будем идти и для каждой страны будем соотносить количество языков, которые упомянул респондент из нее
for country, country_language in zip(data_result, respond_language):
    if isnan(country_language) == True:
        continue
    set_country_language = country_language.split(';')
    set_country_language = list(map(lambda s: s.replace(" ",""), set_country_language))
    for langua in set_country_language:
        tmp_dict[country].append(langua)

tmp_dict

#Пройдемся и с помощью функции Counter вытащим необходимые значения
result_language = pd.Series(str(), index = tmp_dict.keys())

for country in tmp_dict.keys():
    c = Counter(tmp_dict[country])
    l = c.most_common(1)
    if(len(l) != 0):
        result_language[country] = l[0][0]



In [25]:
result_language = result_language.sort_values(ascending = False)
result_language['India']

data_country = data['Country'].value_counts()
data_country['India']

data_frame = pd.DataFrame({'Country_': data_country.index, 'Total': data_country.values}, index = None)
result_language_frame = pd.DataFrame({'Country_': result_language.index, 'Popular': result_language.values}, index = None)
tmp_frame = pd.merge(data_frame, result_language_frame, on='Country_', how='inner')

tmp_frame.head(10)

,Country_,Total,Popular
0,United States,11455,JavaScript
1,India,5197,JavaScript
2,United Kingdom,4395,JavaScript
3,Germany,4143,JavaScript
4,Canada,2233,JavaScript
5,France,1740,JavaScript
6,Poland,1290,JavaScript
7,Australia,913,JavaScript
8,Russian Federation,873,JavaScript
9,Spain,864,JavaScript


In [26]:
tmp_n = tmp_frame[tmp_frame['Popular'] != 'JavaScript']
tmp_n.head(5)

,Country_,Total,Popular
50,South Korea,164,Java
71,Morocco,63,SQL
75,Lebanon,56,SQL
79,Saudi Arabia,52,SQL
81,Malta,46,C#


In [27]:
#Task8
#Меня интересует несколько вопросов
#Почему так популярен язык JavaScrip, почему не Java?
#Касательно самого сета, мне интересно, как связана ученная степень человека и размер компании, в которой он работает, 
#понятно, что люди предпочитают прозрачную систему роста, хороший офис и прочее, но популярны стартапы и какая доля людей
#работает в них?